# Проект  для GoProtect"Стандартизация названий спортивных школ"


### Описание проекта
Сервис  “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.

### Цель
Создать решение для стандартизации названий спортивных школ.
Например одна и та же школа может быть записана по разному
Надо сопоставить эти варианты эталонному названию из предоставленной таблицы

### План проекта:
1. Загрузка и изучение данных
2. Предобработка данных
3. Преобразование тренировочной выборки (аугментация)
4. Обучение модели 
5. Оценка модели
6. Итоговый вывод

## 1. Загрузка и изучение данных

In [1]:
pip install -U sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
#!pip install git+https://github.com/RussianNLP/rutransform

In [3]:
import pandas as pd
import numpy as np
import nltk
import os
import re
import requests
import random

from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import util

from nltk.tokenize import word_tokenize
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sentence_transformers import (
     SentenceTransformer,
     InputExample,
     util,
     losses)

from IPython.display import display

In [4]:
#nltk.download('punkt')

In [5]:
RANDOM_STATE = 201123

In [6]:
file_path = 'C:/Users/no name/Documents/SS/datasets'
if not os.path.exists(file_path):
    file_path = '/content/'
df_train = pd.read_csv(os.path.join(file_path, 'Школы.csv'))
df_test = pd.read_csv(os.path.join(file_path, 'Примерное написание.csv'))    

In [7]:
def basic_info (df):
    print('Первые 5 строк:')
    display(df.head())
    print()
    print('Последние 5 строк:')
    display(df.tail())
    print()
    print('Общая информация по таблице:')
    print()
    df.info()
    print()
    print('Всего явных дубликатов:', df.duplicated().sum())
    print()
    print('Количество пропусков:')
    display(df.isna().sum())
    print()

In [8]:
basic_info(df_train)

Первые 5 строк:


,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург



Последние 5 строк:


,school_id,name,region
301,305,Прогресс,Алтайский край
302,609,"""СШ ""Гвоздика""",Удмуртская республика
303,610,"СШОР ""Надежда Губернии",Саратовская область
304,611,КФК «Айсберг»,Пермский край
305,1836,"ООО ""Триумф""",Москва



Общая информация по таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB

Всего явных дубликатов: 0

Количество пропусков:


school_id    0
name         0
region       0
dtype: int64

In [9]:
basic_info(df_test)

Первые 5 строк:


,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""



Последние 5 строк:


,school_id,name
890,3,"Республика Татарстан, СШОР ФСО Авиатор"
891,3,"СШОР ФСО Авиатор, Республика Татарстан"
892,3,"Республика Татарстан, МБУ ДО СШОР «ФСО ""Авиатор""»"
893,2,"ЯНАО, СШ ""Авангард"""
894,1,"Московская область, СШ ""Авангард"""



Общая информация по таблице:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 895 entries, 0 to 894
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  895 non-null    int64 
 1   name       895 non-null    object
dtypes: int64(1), object(1)
memory usage: 14.1+ KB

Всего явных дубликатов: 0

Количество пропусков:


school_id    0
name         0
dtype: int64

Вывод:
1. Данные представляют собой две таблицы, в одной эталонные наименования спортивных школ, в другой вариации написания в том числе с ошибками.
2. В эталонной таблице всего 306 строк и 3 колонки - id школы, регион, наименование
3. Во второй таблице всего 895 строк и 2 колонки - id школы,  наименование (в части случаев указано с регионом)
4. Эталонную таблицу, используем как тренировачную выборку, другую как тестовую.
5. Пропуски и явные дубликаты в дататсетах не обнаружены.

## 2. Предобработка данных.



Алгоритм будет выявлять соответствие по основным маркерам - само название без аббревиатур и сокращений  + регион, поэтому сначала объединим наименование и регион в одну строку:

In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_train.sort_values(by=['region']))

,school_id,name,region
301,305,Прогресс,Алтайский край
21,22,Беломорец,Архангельская область
65,66,Каскад,Архангельская область
158,159,Созвездие,Астраханская область
50,51,ДЮСШ по ЗВС,Белгородская область
177,178,СШ по ФК,Брянская область
35,36,ГАУ СК,Брянская область
25,26,Брянск,Брянская область
240,241,ФФККВО,Владимирская область
107,108,Мотодром Арена,Владимирская область


In [11]:
# удалим No для обозначения номера
df_train['name'] = df_train['name'].str.replace('No', '')
df_test['name'] = df_test['name'].str.replace('No', '')

In [12]:
df_train['name'] = df_train['name'] + ' ' + df_train ['region']
df_train = df_train.drop(['region'], axis=1)
display(df_train.head())

,school_id,name
0,1,Авангард Московская область
1,2,Авангард Ямало-Ненецкий АО
2,3,Авиатор Республика Татарстан
3,4,Аврора Санкт-Петербург
4,5,Ice Dream / Айс Дрим Санкт-Петербург


In [13]:
def clear(text):
    text = str(text).lower()
    text = re.sub(r'[^А-Яа-яёЁA-Za-zs]', ' ', text)
    text = ' '.join(text.split())
    return text

df_train['name'] = df_train['name'].apply(clear)
df_test['name'] = df_test['name'].apply(clear)

In [14]:
display(df_train.head())
display(df_test.head())

,school_id,name
0,1,авангард московская область
1,2,авангард ямало ненецкий ао
2,3,авиатор республика татарстан
3,4,аврора санкт петербург
4,5,ice dream айс дрим санкт петербург


,school_id,name
0,1836,ооо триумф
1,1836,москва ск триумф
2,610,сшор надежда губернии
3,610,саратовская область гбусо сшор надежда губернии
4,609,сш гвоздика


In [15]:
# словарь с аббревиатурами и расшифровками
abbreviations_dict = {
        'дюсш':'детско юношеская спортивная школа',
        'сдюсшор':'специализированная детско юношеская спортивная школа олимпийского резерва',
        'сш':'спортивная школа',
        'сшор':'спортивная школа олимпийского резерва',
        'ффккр':'федерация фигурного катания на коньках россии',
        'ффк':'федерация фигурного катания',
        'фкк':'фигурного катания на коньках',
        'гау':'государственное автономное учреждение',
        'му':'муниципальное учреждение',
        'мо':'московская область',
        'ао':'автономный округ',
        'ооо':'общество с ограниченной ответственностью',
        'гбусо':'государственное бюджетное учреждение социального обслуживания',
        'уффк':'удмуртская федерация фигурного катания',
        'ффкк':'федерация фигурного катания на коньках',
        'огбу':'областное государственное бюджетное учреждение',
        'рцспзвс':'региональный центр спортивной подготовки по зимним видам спорта',
        'мау':'муниципальное автономное учреждение',
        'маудо':'муниципальное автономное учреждение дополнительного образования.',
        'то':'тульской области',
        'до':' дополнительного образования',
        'оксшор':'областная комплесная спортивная школа олимпийского резерва',
        'гу':'государственное учреждение',
        'звс':'зимним видам спорта',
        'лвс':'летним видам спорта',
        'мбу':'муниципальное бюджетное учреждение',
        'ффккск':'федерация фигурного катания на коньках ставропольского края',
        'мафкк':'московская академия фигурного катания на коньках',
        'нлфк':'национальная лига фигурного катания',
        'ано':'автономная некоммерческая организация ',
        'мбоу':'муниципальное бюджетное общеобразовательное учреждение',
        'ск':'спортивный клуб',
        'сфк':'секция фигурного катания',
        'фк':'фигурного катания',
        'цфксиз':'центр физической культуры, спорта и здоровья',
        'роффкк':'ростовская областная федерация фигурного катания на конках',
        'рсшор':'республиканская спортивная школа олимпийского резерва ',
        'русп':'региональный центр спортивной подготовки',
        'мафсу':'муниципальное автономное физкультурно-спортивное учреждение',
        'мбфсу':'муниципальное бюджетное физкультурно спортивное учреждение',
        'ооффк ио':'общественная организация федерация фигурного катания на коньках в Иркутской области',
        'мку':'муниципальное казенное учреждение',
        'шфк':'школа фигурного катания',
        'цска':'центральный спортивный клуб армии',
        'фау мо рф цска':'федеральное автономное учреждение министерства обороны российской федерации ',
        'рф':'российской федерации',
        'рссш':'республиканская специализированная спортивная школа',
        'роо':'региональная общественная организация',
        'сффк рся':'спортивная федерация фигурного катания на коньках республики саха',
        'кфк':'клуб фигурного катания ',
        'цзвс':'центр зимних видов спорта',
        'гбоу':'государственное бюджетное общеобразовательное учреждение',
        'гбу':'государственное бюджетное учреждение',
        'всшор':'всеволожская спортивная школа олимпийского резерва',
        'цфкис':'центр физической культуры и спорта',
        'уор':'училище олимпийского резерва',
        'рфсоо':'российское физкультурно спортивное общество организация',
        'фок':'физкультурно оздоровительный комплекс',    
        
    }
    
# Функция для замены аббревиатур в тексте на расшифровку
def replace_abbreviations(text):
    words = text.split()
    replaced_words = [abbreviations_dict[word] if word in abbreviations_dict else word for word in words]
    replaced_text = ' '.join(replaced_words)
    return replaced_text

# Замены аббревиатур в колонке 'name' датафрейма df_train
df_train['name'] = df_train['name'].apply(replace_abbreviations)
df_test['name'] = df_test['name'].apply(replace_abbreviations)

display(df_train)

,school_id,name
0,1,авангард московская область
1,2,авангард ямало ненецкий автономный округ
2,3,авиатор республика татарстан
3,4,аврора санкт петербург
4,5,ice dream айс дрим санкт петербург
...,...,...
301,305,прогресс алтайский край
302,609,спортивная школа гвоздика удмуртская республика
303,610,спортивная школа олимпийского резерва надежда ...
304,611,клуб фигурного катания айсберг пермский край


In [16]:
duplicates = df_train[df_train.duplicated(subset=['name'], keep=False)]
display(duplicates)

,school_id,name
96,97,муниципальное автономное физкультурно-спортивн...
97,98,муниципальное автономное физкультурно-спортивн...
169,170,стартайс санкт петербург
185,186,спортивная школа мурманская область
186,187,спортивная школа мурманская область
206,207,спортивная школа олимпийского резерва курганск...
209,210,спортивная школа олимпийского резерва курганск...
285,288,стартайс санкт петербург


Удаляем дубликаты:

In [17]:
df_train = df_train.drop (index=[ 97 , 186 , 209, 285, 67, 38, 25])
df_train.reset_index(drop= True , inplace= True )

In [18]:
df_test = df_test.drop (index=[ 97 , 186 , 209, 285, 67, 38, 25])
df_test.reset_index(drop= True , inplace= True )

In [19]:
with pd.option_context('display.max_rows', None):  
    display(df_train.sort_values(by=['name']))

,school_id,name
4,5,ice dream айс дрим санкт петербург
274,283,proice kids санкт петербург
0,1,авангард московская область
1,2,авангард ямало ненецкий автономный округ
2,3,авиатор республика татарстан
3,4,аврора санкт петербург
282,293,автономная некоммерческая организация лига фи...
287,298,автономная некоммерческая организация спортив...
281,292,автономная некоммерческая организация школа ф...
5,6,айсберг республика крым


Почистим данные:

In [20]:
def remove_non_meaningful(text):
    text = re.sub(r'\b\w{1,1}\b', '', text)  # Удаляем слова длиной менее 2 символов
    text = re.sub(r'\s+', ' ', text).strip()  # Удаляем лишние пробелы
    return text

df_train['name'] = df_train['name'].apply(remove_non_meaningful)
df_test['name'] = df_test['name'].apply(remove_non_meaningful)

In [21]:
display(df_train.head(10))

,school_id,name
0,1,авангард московская область
1,2,авангард ямало ненецкий автономный округ
2,3,авиатор республика татарстан
3,4,аврора санкт петербург
4,5,ice dream айс дрим санкт петербург
5,6,айсберг республика крым
6,7,айсберг рязанская область
7,8,айсберг свердловская область
8,9,звездочка северодвинск
9,10,академия синхронного катания на коньках москва


Для удобства переименуем название колонки в тесте:

In [22]:
df_test = df_test.rename(columns={'name':'name_variant'}).sort_values(by='school_id')
df_test

,school_id,name_variant
887,1,московская область спортивная школа авангард
886,2,янао спортивная школа авангард
884,3,спортивная школа олимпийского резерва фсо авиа...
883,3,республика татарстан спортивная школа олимпийс...
882,3,республика татарстан муниципальное бюджетное у...
...,...,...
4,609,спортивная школа гвоздика
3,610,саратовская область государственное бюджетное ...
2,610,спортивная школа олимпийского резерва надежда ...
1,1836,москва спортивный клуб триумф


## 3. Преобразование тренировочной выборки (аугментация)


Увеличим размер выборки с помощью аугментации для того чтобы в дальнейшем дообучить модель. Напишем функцию, которая сформирует новый датасет путем добавления строк дубликатов с ошибками и переменой слов местами для последующего дообучения модели:

In [23]:
def augment_data(df):
    augmented_data = []

    for i in range(len(df)):
        original_name = df.loc[i, 'name']
        school_id = df.loc[i, 'school_id']

        for _ in range(7):  # Создаем 7 дубликатов с ошибками
            words = original_name.split()
            random_word_index = random.randint(0, len(words) - 1)
            new_words = words.copy()
            error_index = random.randint(0, len(new_words[random_word_index]) - 1)
            new_words[random_word_index] = list(new_words[random_word_index])
            new_words[random_word_index][error_index] = chr(np.random.randint(1072, 1103))
            new_word = ''.join(new_words[random_word_index])
            words[random_word_index] = new_word
            random.shuffle(words)
                                  
            augmented_name = ' '.join(words)
            augmented_data.append({'name': original_name, 'aug_name': augmented_name, 'school_id': school_id})
            
            

    augmented_df = pd.DataFrame(augmented_data)
    return augmented_df

df_train_aug = augment_data(df_train)
df_train_aug = df_train_aug
display(df_train_aug)

,name,aug_name,school_id
0,авангард московская область,обхасть авангард московская,1
1,авангард московская область,авангард москцвская область,1
2,авангард московская область,московская авангагд область,1
3,авангард московская область,область московткая авангард,1
4,авангард московская область,московская обкасть авангард,1
...,...,...,...
2088,общество ограниченной ответственностью триумф ...,общество ответственноатью триумф москва ограни...,1836
2089,общество ограниченной ответственностью триумф ...,москва ответственностью ограниченной тлиумф об...,1836
2090,общество ограниченной ответственностью триумф ...,обществг триумф ограниченной ответственностью ...,1836
2091,общество ограниченной ответственностью триумф ...,обществы ответственностью ограниченной триумф ...,1836


In [24]:
df_train_aug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093 entries, 0 to 2092
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       2093 non-null   object
 1   aug_name   2093 non-null   object
 2   school_id  2093 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 49.2+ KB


## 4. Обучение модели


### Модель №1: (sentence-transformers) LaBSE

In [25]:
model_1 = SentenceTransformer('sentence-transformers/LaBSE')

In [26]:
corpuse_embedding_1 = model_1.encode(df_train.name.values)

In [27]:
corpuse_embedding_1

array([[ 0.04227928,  0.02789917,  0.02638606, ..., -0.03887578,
        -0.06346413, -0.05755835],
       [ 0.03132331,  0.01584869,  0.04261846, ..., -0.04308641,
        -0.03137583, -0.01525755],
       [ 0.01162198, -0.03974545,  0.00175533, ...,  0.02332445,
         0.06898228,  0.05103588],
       ...,
       [ 0.0181281 , -0.03543217,  0.01988792, ..., -0.01726287,
         0.02678434, -0.03435838],
       [ 0.06397197,  0.01384863,  0.0186362 , ..., -0.01232259,
         0.03450641, -0.00033415],
       [-0.00088473, -0.00694319,  0.03504249, ..., -0.04329306,
         0.0427042 , -0.03005286]], dtype=float32)

In [28]:
query_embedding_1 = model_1.encode(df_test.name_variant.values)

In [29]:
query_embedding_1

array([[ 0.03133927,  0.01080929,  0.00775494, ..., -0.04476377,
        -0.04485656, -0.05191057],
       [ 0.00269639,  0.0041837 ,  0.00678373, ..., -0.02626428,
        -0.06016866, -0.0615179 ],
       [ 0.00240553, -0.04714845, -0.0313983 , ..., -0.02556991,
         0.01677923,  0.03900205],
       ...,
       [ 0.00153687, -0.00443714,  0.01859644, ..., -0.01346344,
         0.00155952, -0.02998321],
       [ 0.02594607,  0.01444612,  0.01639306, ..., -0.04595377,
        -0.01468683, -0.03568109],
       [-0.00775135, -0.00167825,  0.04449824, ..., -0.00327478,
         0.04104581, -0.04078319]], dtype=float32)

In [30]:
result = util.semantic_search(query_embedding_1, corpuse_embedding_1, top_k=3)

In [31]:
result

[[{'corpus_id': 0, 'score': 0.787702739238739},
  {'corpus_id': 178, 'score': 0.7783902883529663},
  {'corpus_id': 204, 'score': 0.768876314163208}],
 [{'corpus_id': 168, 'score': 0.6131557822227478},
  {'corpus_id': 178, 'score': 0.6087374091148376},
  {'corpus_id': 55, 'score': 0.5879767537117004}],
 [{'corpus_id': 195, 'score': 0.8954892158508301},
  {'corpus_id': 138, 'score': 0.8549620509147644},
  {'corpus_id': 199, 'score': 0.7830220460891724}],
 [{'corpus_id': 195, 'score': 0.9000043272972107},
  {'corpus_id': 138, 'score': 0.8760614395141602},
  {'corpus_id': 199, 'score': 0.7711621522903442}],
 [{'corpus_id': 138, 'score': 0.8316155076026917},
  {'corpus_id': 195, 'score': 0.8158401846885681},
  {'corpus_id': 94, 'score': 0.7448347210884094}],
 [{'corpus_id': 138, 'score': 0.850380539894104},
  {'corpus_id': 195, 'score': 0.8324564695358276},
  {'corpus_id': 94, 'score': 0.7663852572441101}],
 [{'corpus_id': 138, 'score': 0.8321430087089539},
  {'corpus_id': 195, 'score': 0.8

## 5. Оценка модели

In [32]:
def best_result(df):
    i = sorted(df, key=lambda d: d['score'])
    return i[-1]['corpus_id']

In [33]:
df_test['candidate_idx'] = [best_result(x) for x in result]
df_test['candidate_school_id'] = df_train.school_id.values[df_test.candidate_idx.values]
df_test['candidate_name'] = df_test.name_variant.values[df_test.candidate_idx.values]
display(df_test)

accuracy = round((df_test.school_id == df_test.candidate_school_id).sum()/df_test.shape[0], 2)
print(f"Accuracy: {accuracy:.2f}%")

,school_id,name_variant,candidate_idx,candidate_school_id,candidate_name
887,1,московская область спортивная школа авангард,0,1,московская область спортивная школа авангард
886,2,янао спортивная школа авангард,168,173,курганская область детско юношеская спортивная...
884,3,спортивная школа олимпийского резерва фсо авиа...,195,201,спб государственное бюджетное учреждение спорт...
883,3,республика татарстан спортивная школа олимпийс...,195,201,спб государственное бюджетное учреждение спорт...
882,3,республика татарстан муниципальное бюджетное у...,138,143,санкт петербург спб гбпоу алвс динамо санкт пе...
...,...,...,...,...,...
4,609,спортивная школа гвоздика,295,609,московская область муниципальное автономное уч...
3,610,саратовская область государственное бюджетное ...,296,610,московская область муниципальное автономное уч...
2,610,спортивная школа олимпийского резерва надежда ...,296,610,московская область муниципальное автономное уч...
1,1836,москва спортивный клуб триумф,244,251,москва государственное бюджетное учреждение мо...


Accuracy: 0.41%


## 6. Итоговый вывод

1. Данные представляют собой две таблицы, в одной эталонные наименования спортивных школ, в другой вариации написания в том числе с ошибками.
2. В эталонной таблице всего 306 строк и 3 колонки - id школы, регион, наименование
3. Во второй таблице всего 895 строк и 2 колонки - id школы,  наименование (в части случаев указано с регионом)
4. Эталонную таблицу, используем как тренировачную выборку, другую как тестовую.
5. Пропуски  в дататсетах не обнаружены.
6. При обработке данных выявили дубликаты и удалили их. 
7. В тренировочный датасет добавлен признак - наименование с расшифрованной аббревиатурой 
8. Все наименования приведены к нижнему регистру, удалены ненужные символы, пробелы и знаки
9. На тренировочной выборке проведена аугментация путем побавления 7 дубликатов с ошибками и перемены слов местами для того чтобы в дальнейшем дообучить модель на этих данных.
10. Протестирована модель sentence-transformers - LaBSE
11. Получен результат (доля правильно определенных наименований) Accuracy: 
12. Результат нуждается в улучшении, можно попробовать дообучить модель на тренировочных данных, увеличить количество признаков (добавить колонки с разными вариантами написаний названий школ) , потестировать другие модели.
13. Рекомендация заказчику: в эталонном датасете присутствуют опечатки, написание одних и тех же названий в разном регистре, чего не должно быть - при обработке данных выявили дубликаты и удалили их. Но наверняка этим дубликатам не просто так присвоили уникальные id - наверняка по ним сформированы заявки и т д, рекомендуется ручная проверка эталонных данных.
